# Keras DSVM Deep Learning PaaS


## Introduction

This recipe shows how to run Keras using Batch AI on DSVM. DSVM supports tensorflow, cntk and theano backends for running Keras. Currently only tensorflow and cntk backends supports running on GPU.

## Details

- DSVM has Keras framework preinstalled;
- 셈플코드는 Glove + bidirectional LSTM 으로 작성된 간단한 Text Classification 모델을 이용합니다.
- The script downloads the standard MNIST Database on its own;
- Standard output of the job will be stored on Azure File Share.

## Instructions

### Install Dependencies and Create Configuration file.
Follow [instructions](/recipes) to install all dependencies and create configuration file.

### Read Configuration and Create Batch AI client

In [1]:
from __future__ import print_function

import time
from datetime import datetime
import os
import sys
import zipfile

from azure.storage.file import FileService, FilePermissions
import azure.mgmt.batchai.models as models

# utilities.py contains helper functions used by different notebooks
sys.path.append('./')
import utilities

cfg = utilities.Configuration('configuration.json')
client = utilities.create_batchai_client(cfg)

### Create File Share

For this example we will create a new File Share with name `batchaidsvmsample` under your storage account.

**Note** You don't need to create new file share for every cluster. We are doing this in this sample to simplify resource management for you.

In [3]:
azure_file_share_name = 'kimhoondong352'
service = FileService(cfg.storage_account_name, cfg.storage_account_key)
service.create_share(azure_file_share_name, fail_on_exist=False)

True

### Configure Compute Cluster

- For this example we will use a gpu cluster of `STANDARD_NC6` nodes. Number of nodes in the cluster is configured with `nodes_count` variable;
- We will mount file share at folder with name `external`. Full path of this folder on a computer node will be `$AZ_BATCHAI_MOUNT_ROOT/external`;
- We will call the cluster `nc6`;


So, the cluster will have the following parameters:

In [4]:
azure_file_share = 'external'
nodes_count = 1
cluster_name = 'nc6-352'

volumes = models.MountVolumes(
    azure_file_shares=[
        models.AzureFileShareReference(
            account_name=cfg.storage_account_name,
            credentials=models.AzureStorageCredentialsInfo(
                account_key=cfg.storage_account_key),
            azure_file_url = 'https://{0}.file.core.windows.net/{1}'.format(
                cfg.storage_account_name, azure_file_share_name),
            relative_mount_path=azure_file_share)
    ]
)

parameters = models.ClusterCreateParameters(
    location=cfg.location,
    vm_size="STANDARD_NC6",
    virtual_machine_configuration=models.VirtualMachineConfiguration(
        image_reference=models.ImageReference(
            publisher="microsoft-ads",
            offer="linux-data-science-vm-ubuntu",
            sku="linuxdsvmubuntu",
            version="latest")),
    scale_settings=models.ScaleSettings(
        manual=models.ManualScaleSettings(target_node_count=nodes_count)
    ),
    node_setup=models.NodeSetup(
        mount_volumes=volumes
    ),
    user_account_settings=models.UserAccountSettings(
        admin_user_name=cfg.admin,
        admin_user_password=cfg.admin_password,
        admin_user_ssh_public_key=cfg.admin_ssh_key
    )
)

### Create Compute Cluster

In [5]:
_ = client.clusters.create(cfg.resource_group, cluster_name, parameters)

### Monitor Cluster Creation

utilities.py contains a helper function allowing to wait for the cluster to become available - all nodes are allocated and finished preparation.

In [6]:
cluster = client.clusters.get(cfg.resource_group, cluster_name)
utilities.print_cluster_status(cluster)

Cluster state: AllocationState.steady Target: 1; Allocated: 1; Idle: 1; Unusable: 0; Running: 0; Preparing: 0; Leaving: 0


### Deploy Sample Script and Configure the Input Directories


The job will be able to reference those directories using ```$AZ_BATCHAI_INPUT_SCRIPT``` environment variable.

In [8]:
script_directory = 'trainall_script'
#script_file = 'GloveBidirectionalLSTM_v3_1.py'
script_file = 'BidirectionalLSTM_v2_1_1host.py'

#dataset_file1 = 'glove.6B.100d.txt'
#dataset_file2 = 'labeledTrainData.tsv'

service = FileService(cfg.storage_account_name, cfg.storage_account_key)
service.create_directory(
    azure_file_share_name, script_directory, fail_on_exist=False)

#Script file
service.create_file_from_path(
    azure_file_share_name, script_directory, script_file, script_file)

#Dataset file
#service.create_file_from_path(
#    azure_file_share_name, script_directory, dataset_file1, dataset_file1)
#service.create_file_from_path(
#    azure_file_share_name, script_directory, dataset_file2, dataset_file2)

In [9]:
input_directories = [
    models.InputDirectory(
        id='SCRIPTS',
        path='$AZ_BATCHAI_MOUNT_ROOT/{0}/{1}'.format(azure_file_share, script_directory))
]

### Configure Output Directories
We will store standard and error output of the job in File Share:

In [10]:
std_output_path_prefix = "$AZ_BATCHAI_MOUNT_ROOT/{0}".format(azure_file_share)

### Configure Job

- Will use configured previously input and output directories;
- Will run standard `mnist_cnn.py` from SCRIPT input directory using custom framework;
- Keral will use theano backend; DSVM supports cntk, tensorflow and theano backends for keral, just change KERAS_BACKEND to "tensorflow" or "theano" to use corresponding backend. Note, theano backend will run on CPU. 
- Will output standard output and error streams to file share.


In [11]:
job_name = datetime.utcnow().strftime("azure_batch_ai_keras_351_%m_%d_%Y_%H%M%S")
parameters = models.job_create_parameters.JobCreateParameters(
     location=cfg.location,
     cluster=models.ResourceId(cluster.id),
     node_count=nodes_count,
     input_directories=input_directories,
     std_out_err_path_prefix=std_output_path_prefix,
     custom_toolkit_settings = models.CustomToolkitSettings(
         command_line='KERAS_BACKEND=tensorflow python $AZ_BATCHAI_INPUT_SCRIPTS/'+script_file))

In [38]:
## /mnt/batch/tasks/shared/LS_root/mounts/external/trainall_script/

### Create a training Job and wait for Job completion


In [12]:
_ = client.jobs.create(cfg.resource_group, job_name, parameters)
print('Created Job: {}'.format(job_name))

Created Job: azure_batch_ai_keras_351_01_27_2018_082235


### Wait for Job to Finish
The job will start running when the cluster will have enought idle nodes. The following code waits for job to start running printing the cluster state. During job run, the code prints current content of stdout.txt.

**Note** Execution may take several minutes to complete.

In [13]:
utilities.wait_for_job_completion(client, cfg.resource_group, job_name, cluster_name, 'stdouterr', 'stdout.txt')

Cluster state: AllocationState.steady Target: 1; Allocated: 1; Idle: 0; Unusable: 0; Running: 1; Preparing: 0; Leaving: 0
Job state: running ExitCode: None
Waiting for job output to become available...
Loading data...

17465344/17464789 [==============================] - 0s 0us/step
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 100)
x_test shape: (25000, 100)
Build model...
Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/4

24960/25000 [============================>.] - ETA: 0s - loss: 0.4245 - acc: 0.8048


25000/25000 [==============================] - 274s 11ms/step - loss: 0.4242 - acc: 0.8050 - val_loss: 0.3468 - val_acc: 0.8495
Epoch 2/4

 3232/25000 [==>...........................] - ETA: 3:07 - loss: 0.2225 - acc: 0.9226


 6272/25000 [======>.......................] - ETA: 2:42 - loss: 0.2209 - acc: 0.9188


 9408/25000 [==========>...................] - ETA: 2:15 - loss: 0.2198 - acc: 0.9193


12736/25000 [==============>...............] - ETA: 1:46 - loss: 0.2249 - acc: 0.9168


...

25000/25000 [==============================] - 267s 11ms/step - loss: 0.0858 - acc: 0.9701 - val_loss: 0.7088 - val_acc: 0.8237

 3520/25000 [===>..........................] - ETA: 44s


 9568/25000 [==========>...................] - ETA: 32s


15648/25000 [=================>............] - ETA: 19s


21440/25000 [========================>.....] - ETA: 7s


25000/25000 [==============================] - 53s 2ms/step
Test score: 0.708789501987
Test accuracy: 0.82368
Job state: succeeded ExitCode: 0


### Download stdout.txt and stderr.txt files for the Job

In [67]:
files = client.jobs.list_output_files(cfg.resource_group, job_name, models.JobsListOutputFilesOptions("stdOuterr")) 
for file in list(files):
    utilities.download_file(file.download_url, file.name)
print("All files Downloaded")

All files Downloaded


In [68]:
print('stdout.txt content:')
with open('stdout.txt') as f:
    print(f.read())

print('stderr.txt content:')
with open('stderr.txt') as f:
    print(f.read())

stdout.txt content:
Loading data...

17465344/17464789 [==============================] - 0s 0us/step
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 100)
x_test shape: (25000, 100)
Build model...
Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/4

25000/25000 [==============================] - 279s 11ms/step - loss: 0.4072 - acc: 0.8119 - val_loss: 0.3496 - val_acc: 0.8470
Epoch 2/4

25000/25000 [==============================] - 264s 11ms/step - loss: 0.2325 - acc: 0.9092 - val_loss: 0.3658 - val_acc: 0.8412
Epoch 3/4

25000/25000 [==============================] - 259s 10ms/step - loss: 0.1299 - acc: 0.9514 - val_loss: 0.4169 - val_acc: 0.8372
Epoch 4/4

25000/25000 [==============================] - 255s 10ms/step - loss: 0.0749 - acc: 0.9749 - val_loss: 0.5498 - val_acc: 0.8360

25000/25000 [==============================] - 52s 2ms/step
Test score: 0.549846779194
Test accuracy: 0.83604

stderr.txt content:
bash: /

### Delete the Job

In [51]:
client.jobs.delete(cfg.resource_group, job_name)

### Delete the Cluster
When you are finished with the sample and don't want to submit any more jobs you can delete the cluster using the following code.

In [52]:
client.clusters.delete(cfg.resource_group, cluster_name)

### Delete File Share
When you are finished with the sample and don't want to submit any more jobs you can delete the file share completely with all files using the following code.

In [53]:
service = FileService(cfg.storage_account_name, cfg.storage_account_key)
service.delete_share(azure_file_share_name)

True

Token expired or is invalid. Attempting to refresh.
